In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

data_path = "test2.xlsx"
property_data = pd.read_excel(data_path)

property_data["date"] = pd.to_datetime(property_data["date"])

In [2]:
property_data["yr_sold"] = property_data["date"].dt.year
property_data["house_age"] = property_data["yr_sold"] - np.where(property_data["yr_renovated"] > 0,
    property_data["yr_renovated"],property_data["yr_built"])
cols_to_drop = ["date", "yr_built", "yr_renovated"]
property_data = property_data.drop(columns=cols_to_drop)

"house_age" is a better contributer to "grade" and takes into account when the house was reinvented as well.

In [ ]:

property_data = property_data.drop_duplicates(subset=["id"]).reset_index(drop=True)
print("Rows after removing duplicates:", len(property_data))

property_data = property_data.drop(columns=["zipcode"])


Duplicate property records were removed to ensure a unique mapping between tabular entries and satellite imagery. Zipcode was excluded due to low correlation with price and redundancy with continuous geospatial features. "id" was retained solely for mapping tabular data and images but is not included while modelling.

In [4]:
property_data["log_sqft_lot"] = np.log1p(property_data["sqft_lot"])
property_data = property_data.drop(columns=["sqft_lot"])


In [5]:
property_data["scenic_score"] = property_data["view"] + 2 * property_data["waterfront"]
property_data = property_data.drop(columns=["view", "waterfront"])


In [ ]:
non_features = ["id"]

X_tab = property_data.drop(columns=non_features) \
                     .select_dtypes(include=[np.number]) \
                     .fillna(0) \
                     .values

print("Final feature shape:", X_tab.shape)

In [ ]:
excel_path = "processed_housing_dataset.xlsx"
property_data.to_excel(excel_path, index=False)

print("Excel file created:", excel_path)